# Install & Imports

In [ ]:
!pip install -q ipywidgets groq
import base64
import time
import json
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output, HTML
import ipywidgets as widgets
from groq import Groq
from google.colab import userdata

# API Setup

In [ ]:

import os
from groq import Groq

try:
    API_KEY = os.environ.get("GROQ_API_KEY")
    if not API_KEY:
        raise ValueError("GROQ_API_KEY environment variable not set")

    client = Groq(api_key=API_KEY)
    print("Groq client ready ✓")

except Exception as e:
    print("Error loading API key:", e)


Groq client ready ✓


# 1. FIXED: JavaScript Rendering (No Broken Images)

In [ ]:
def render_mermaid_html(clean_code, index):
    """
    Renders Mermaid locally using JavaScript (Mermaid.js) to avoid broken images.
    Includes a fallback link to mermaid.ink.
    """
    try:
        # Generate fallback URL
        graphbytes = clean_code.encode("utf-8")
        base64_bytes = base64.b64encode(graphbytes)
        base64_string = base64_bytes.decode("ascii")
        img_url = "https://mermaid.ink/img/" + base64_string

        # Unique ID to ensure scripts target the correct div
        div_id = f"mermaid-diagram-{index}"

        # HTML + JS Injection
        html_block = f"""
        <div style="border: 1px solid #e0e0e0; border-radius: 8px; padding: 15px; margin-bottom: 25px; background: #fff; box-shadow: 0 4px 6px rgba(0,0,0,0.05);">
            <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 10px; border-bottom: 1px solid #f0f0f0; padding-bottom: 8px;">
                <span style="font-weight: bold; color: #555; font-family: sans-serif;">Diagram #{index}</span>
                <a href="{img_url}" target="_blank" style="text-decoration: none; color: #007bff; font-family: sans-serif; font-size: 0.85em; background: #f0f8ff; padding: 4px 8px; border-radius: 4px;">
                    OPEN IMAGE ↗
                </a>
            </div>

            <div class="mermaid" id="{div_id}" style="text-align: center; overflow-x: auto;">
                {clean_code}
            </div>
        </div>

        <script>
            (function() {{
                var targetId = "#{div_id}";
                function render() {{
                    if (window.mermaid) {{
                        mermaid.init(undefined, document.querySelectorAll(targetId));
                    }} else {{
                        setTimeout(render, 100);
                    }}
                }}

                if (typeof mermaid === 'undefined') {{
                    var script = document.createElement('script');
                    script.src = "https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.min.js";
                    script.onload = function() {{
                        mermaid.initialize({{ startOnLoad: false, theme: 'default' }});
                        render();
                    }};
                    document.head.appendChild(script);
                }} else {{
                    render();
                }}
            }})();
        </script>
        """
        display(HTML(html_block))
        return True
    except Exception as e:
        print(f"❌ Render Error for Diagram {index}: {e}")
        return False

# DEBUG LOGIC

In [ ]:
debug_checkbox = widgets.Checkbox(value=True, description='Show Debug Logs & Source Code')

def debug_log(message, title="DEBUG"):
    if debug_checkbox.value:
        display(HTML(f"<div style='background:#f8f9fa; padding:5px; border-left: 3px solid #f66e09; color:#666; font-family:monospace; font-size:11px; margin: 2px 0;'><b>[{title}]</b> {message}</div>"))

def display_collapsible(title, content, style="info"):
    colors = {"info": "#e9ecef", "raw": "#ffeeba", "prompt": "#d1e7dd"}
    bg = colors.get(style, "#f0f0f0")
    content_escaped = str(content).replace("<", "&lt;").replace(">", "&gt;")
    html = f"""
    <details style="background-color: {bg}; padding: 8px; border-radius: 4px; margin-bottom: 8px; border: 1px solid #dee2e6;">
        <summary style="cursor: pointer; font-weight: bold; font-family: sans-serif; font-size: 0.9em;">{title}</summary>
        <pre style="white-space: pre-wrap; margin-top: 5px; font-size: 0.8em; color: #333; background:rgba(255,255,255,0.5); padding:5px;">{content_escaped}</pre>
    </details>
    """
    display(HTML(html))

# Prompts

In [ ]:
FIRST_ADD_PROMPT_TEMPLATE = """
You are an expert software architect.
System: {system_desc}
Attributes: {quality_attrs}

Perform ADD Iteration 1.
Output strictly in markdown:
## Architectural Drivers
## Proposed Tactics & Patterns
## Architecture Proposal
## Trade-offs & Risks
## Design Decisions
"""

MERMAID_PROMPT_TEMPLATE = """
You are a technical diagram expert. Generate 5 Mermaid diagrams for:
System: {system_desc}
Attributes: {quality_attrs}

Output 5 separate ```mermaid code blocks.

1. System Context: Use `graph TB`. Show User -> System -> External Systems.
2. Container Diagram: Use `graph TB`. Show Web App, API, Database, Mobile App.
3. Component Diagram: Use `graph LR` with `subgraph`. Show internal modules of the API.
4. Deployment Diagram: Use `graph TB` with `subgraph` for Nodes (e.g., "AWS Cloud", "Docker").
5. Sequence Diagram: Use `sequenceDiagram`. Show a critical user scenario (e.g. Login or Checkout).

CRITICAL SYNTAX RULES:
- For Diagrams 1, 2, 3, 4: Use ONLY standard flowchart syntax (`graph TB` or `graph LR`).
- DO NOT use `C4Context` or specialized classes (they break easily).
- DO NOT use `participant` inside a `graph` block.
- For Diagram 5: Use ONLY `sequenceDiagram` with `participant` or `actor`.
- Use simple node IDs like A, B, C with labels in brackets like A[User].
- Avoid special characters in labels (no quotes, parentheses in text).

Begin.
"""

# Logic Functions (Using Llama on Groq)

In [ ]:
def generate_add_design(system_desc, quality_attrs):
    if not system_desc.strip() or not quality_attrs.strip():
        return "⚠️ Please fill in both inputs.", "N/A"

    prompt = FIRST_ADD_PROMPT_TEMPLATE.format(system_desc=system_desc, quality_attrs=quality_attrs)
    debug_log("Calling Groq (Llama 3) for ADD...", "Step 1")

    try:
        # 🔧 CHANGE: Using llama-3.3-70b-versatile
        resp = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1500,
            temperature=0.6
        )
        return resp.choices[0].message.content.strip(), prompt
    except Exception as e:
        return f"❌ Error calling Groq API: {e}", prompt

def generate_mermaid_diagrams(system_desc, quality_attrs):
    prompt = MERMAID_PROMPT_TEMPLATE.format(
        system_desc=system_desc,
        quality_attrs=quality_attrs
    )
    debug_log("Calling Groq (Llama 3) for Mermaid...", "Step 2")

    try:
        # 🔧 CHANGE: Using llama-3.3-70b-versatile
        resp = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=2500,
            temperature=0.3
        )
        return resp.choices[0].message.content.strip(), prompt
    except Exception as e:
        return f"❌ Error calling Groq API: {e}", prompt

def process_mermaid_response(full_response):
    debug_log("Parsing response for mermaid blocks...", "Parser")

    if "Error" in full_response:
        print(full_response)
        return

    parts = full_response.split("```mermaid")
    count = 0

    if len(parts) < 2:
        print("❌ No mermaid blocks found in response.")
        print("📄 Raw response preview:")
        display_collapsible("Full LLM Response", full_response[:2000], "raw")
        return

    for part in parts[1:]:
        if "```" in part:
            code = part.split("```")[0].strip()
        else:
            code = part.strip()

        lines = []
        for line in code.splitlines():
            stripped = line.strip()
            if stripped and not stripped.startswith('%%') and not stripped.startswith('Here'):
                lines.append(line)

        if not lines:
            continue

        clean_code = "\n".join(lines)
        count += 1

        if debug_checkbox.value:
            display_collapsible(f"🔎 Source Code: Diagram {count}", clean_code, "raw")

        render_mermaid_html(clean_code, count)

    if count == 0:
        print("❌ Could not extract any valid mermaid diagrams.")
    else:
        print(f"✅ Rendered {count} diagram(s)")


# UI LAYOUT | Grok Version

In [ ]:
title = widgets.HTML("<h2>🛠️ ADD + Mermaid Generator (Groq + Llama)</h2>")
desc_input = widgets.Textarea(
    placeholder="E.g., A ride-sharing app that connects drivers with passengers...",
    layout={'width': '98%', 'height': '80px'}
)
qa_input = widgets.Text(
    placeholder="E.g., Availability, Low Latency, Scalability...",
    layout={'width': '98%'}
)
btn_go = widgets.Button(
    description="🚀 Generate Architecture",
    button_style='primary',
    layout={'width': '220px'}
)
output = widgets.Output(layout={'border': '1px solid #ccc', 'padding': '15px', 'margin_top': '10px'})

def on_go_clicked(b):
    with output:
        clear_output()

        if not desc_input.value.strip() or not qa_input.value.strip():
            print("⚠️ Please fill in all fields.")
            return

        # 1. Generate ADD
        display(Markdown("### 📐 Step 1: Generating ADD Architecture..."))
        add_text, add_prompt = generate_add_design(desc_input.value, qa_input.value)
        if debug_checkbox.value: display_collapsible("Prompt", add_prompt, "prompt")
        display(Markdown(add_text))
        display(Markdown("---"))

        # 2. Generate Mermaid Diagrams
        display(Markdown("### 📊 Step 2: Generating Diagrams..."))
        mermaid_text, mermaid_prompt = generate_mermaid_diagrams(desc_input.value, qa_input.value)
        if debug_checkbox.value: display_collapsible("Prompt", mermaid_prompt, "prompt")
        process_mermaid_response(mermaid_text)

btn_go.on_click(on_go_clicked)

display(widgets.VBox([
    title,
    widgets.Label("📝 System Description:"),
    desc_input,
    widgets.Label("⚡ Quality Attributes (comma-separated):"),
    qa_input,
    debug_checkbox,
    widgets.HBox([btn_go], layout={'justify_content': 'center', 'margin': '15px 0'}),
    output
], layout={'padding': '20px', 'max_width': '900px', 'margin': 'auto'}))

#OPENAI VERSION V1

# Install & Imports

In [ ]:
# First, install openai if needed
# !pip install openai

import base64
import time
import json
from IPython.display import display, Markdown, clear_output, HTML
import ipywidgets as widgets

from openai import OpenAI
from google.colab import userdata

# API Setup

In [ ]:
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    client = OpenAI(api_key=OPENAI_API_KEY)
except Exception as e:
    print("❌ Error retrieving API Key. Ensure 'OPENAI_API_KEY' is in Colab Secrets.")
    # Fallback for manual key entry if secrets fail
    # client = OpenAI(api_key="sk-...")

#  2. FIXED: JavaScript Rendering (No Broken Images)

In [ ]:
def render_mermaid_html(clean_code, index):
    """
    Renders Mermaid locally using JavaScript (Mermaid.js) to avoid broken images.
    Includes a fallback link to mermaid.ink.
    """
    try:
        # Generate fallback URL (still useful for "Open Image")
        graphbytes = clean_code.encode("utf-8")
        base64_bytes = base64.b64encode(graphbytes)
        base64_string = base64_bytes.decode("ascii")
        img_url = "https://mermaid.ink/img/" + base64_string

        # Unique ID to ensure scripts target the correct div
        div_id = f"mermaid-diagram-{index}"

        # HTML + JS Injection
        # We inject the library if missing, then render the specific div
        html_block = f"""
        <div style="border: 1px solid #e0e0e0; border-radius: 8px; padding: 15px; margin-bottom: 25px; background: #fff; box-shadow: 0 4px 6px rgba(0,0,0,0.05);">
            <div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 10px; border-bottom: 1px solid #f0f0f0; padding-bottom: 8px;">
                <span style="font-weight: bold; color: #555; font-family: sans-serif;">Diagram #{index}</span>
                <a href="{img_url}" target="_blank" style="text-decoration: none; color: #007bff; font-family: sans-serif; font-size: 0.85em; background: #f0f8ff; padding: 4px 8px; border-radius: 4px;">
                    OPEN IMAGE ↗
                </a>
            </div>

            <div class="mermaid" id="{div_id}" style="text-align: center; overflow-x: auto;">
                {clean_code}
            </div>
        </div>

        <script>
            // specific function to load and render
            (function() {{
                var targetId = "#{div_id}";

                function render() {{
                    if (window.mermaid) {{
                        mermaid.init(undefined, document.querySelectorAll(targetId));
                    }} else {{
                        // Retry if library isn't ready yet
                        setTimeout(render, 100);
                    }}
                }}

                if (typeof mermaid === 'undefined') {{
                    // Load mermaid.js from CDN if not already loaded
                    var script = document.createElement('script');
                    script.src = "https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.min.js";
                    script.onload = function() {{
                        mermaid.initialize({{ startOnLoad: false, theme: 'default' }});
                        render();
                    }};
                    document.head.appendChild(script);
                }} else {{
                    render();
                }}
            }})();
        </script>
        """
        display(HTML(html_block))
        return True
    except Exception as e:
        print(f"❌ Render Error for Diagram {index}: {e}")
        return False


# 3. Debug Logic

In [ ]:
debug_checkbox = widgets.Checkbox(value=True, description='Show Debug Logs & Source Code')

def debug_log(message, title="DEBUG"):
    if debug_checkbox.value:
        display(HTML(f"<div style='background:#f8f9fa; padding:5px; border-left: 3px solid #198754; color:#666; font-family:monospace; font-size:11px; margin: 2px 0;'><b>[{title}]</b> {message}</div>"))

def display_collapsible(title, content, style="info"):
    colors = {"info": "#e9ecef", "raw": "#ffeeba", "prompt": "#d1e7dd"}
    bg = colors.get(style, "#f0f0f0")
    content_escaped = str(content).replace("<", "&lt;").replace(">", "&gt;")
    html = f"""
    <details style="background-color: {bg}; padding: 8px; border-radius: 4px; margin-bottom: 8px; border: 1px solid #dee2e6;">
        <summary style="cursor: pointer; font-weight: bold; font-family: sans-serif; font-size: 0.9em;">{title}</summary>
        <pre style="white-space: pre-wrap; margin-top: 5px; font-size: 0.8em; color: #333; background:rgba(255,255,255,0.5); padding:5px;">{content_escaped}</pre>
    </details>
    """
    display(HTML(html))

# 4. Prompts

In [ ]:
FIRST_ADD_PROMPT_TEMPLATE = """
You are an expert software architect.
System: {system_desc}
Attributes: {quality_attrs}

Perform ADD Iteration 1.
Output strictly in markdown:
## Architectural Drivers
## Proposed Tactics & Patterns
## Architecture Proposal
## Trade-offs & Risks
## Design Decisions
"""

MERMAID_PROMPT_TEMPLATE = """
You are a technical diagram expert. Generate 5 Mermaid diagrams for:
System: {system_desc}
Attributes: {quality_attrs}

Output 5 separate ```mermaid code blocks.

1. System Context: Use `graph TB`. Show User -> System -> External Systems.
2. Container Diagram: Use `graph TB`. Show Web App, API, Database, Mobile App.
3. Component Diagram: Use `graph LR` with `subgraph`. Show internal modules of the API.
4. Deployment Diagram: Use `graph TB` with `subgraph` for Nodes (e.g., "AWS Cloud", "Docker").
5. Sequence Diagram: Use `sequenceDiagram`. Show a critical user scenario (e.g. Login or Checkout).

CRITICAL SYNTAX RULES:
- For Diagrams 1, 2, 3, 4: Use ONLY standard flowchart syntax (`graph TB` or `graph LR`).
- DO NOT use `C4Context` or specialized classes (they break easily).
- DO NOT use `participant` inside a `graph` block.
- For Diagram 5: Use ONLY `sequenceDiagram` with `participant` or `actor`.
- Use simple node IDs like A, B, C with labels in brackets like A[User].
- Avoid special characters in labels (no quotes, parentheses in text).

Begin.
"""

# 5. Logic Functions | OpenAI(GPT4o)

In [ ]:
def generate_add_design(system_desc, quality_attrs):
    if not system_desc.strip() or not quality_attrs.strip():
        return "⚠️ Please fill in both inputs.", "N/A"

    prompt = FIRST_ADD_PROMPT_TEMPLATE.format(system_desc=system_desc, quality_attrs=quality_attrs)
    debug_log("Calling OpenAI for ADD...", "Step 1")

    try:
        resp = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1500,
            temperature=0.6
        )
        return resp.choices[0].message.content.strip(), prompt
    except Exception as e:
        return f"❌ Error calling OpenAI API: {e}", prompt

def generate_mermaid_diagrams(system_desc, quality_attrs):
    prompt = MERMAID_PROMPT_TEMPLATE.format(
        system_desc=system_desc,
        quality_attrs=quality_attrs
    )
    debug_log("Calling OpenAI for Mermaid...", "Step 2")

    try:
        resp = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=2500,
            temperature=0.3
        )
        return resp.choices[0].message.content.strip(), prompt
    except Exception as e:
        return f"❌ Error calling OpenAI API: {e}", prompt

def process_mermaid_response(full_response):
    debug_log("Parsing response for mermaid blocks...", "Parser")

    if "Error" in full_response:
        print(full_response)
        return

    parts = full_response.split("```mermaid")
    count = 0

    if len(parts) < 2:
        print("❌ No mermaid blocks found in response.")
        print("📄 Raw response preview:")
        display_collapsible("Full LLM Response", full_response[:2000], "raw")
        return

    for part in parts[1:]:
        if "```" in part:
            code = part.split("```")[0].strip()
        else:
            code = part.strip()

        lines = []
        for line in code.splitlines():
            stripped = line.strip()
            if stripped and not stripped.startswith('%%') and not stripped.startswith('Here'):
                lines.append(line)

        if not lines:
            continue

        clean_code = "\n".join(lines)
        count += 1

        if debug_checkbox.value:
            display_collapsible(f"🔎 Source Code: Diagram {count}", clean_code, "raw")

        render_mermaid_html(clean_code, count)

    if count == 0:
        print("❌ Could not extract any valid mermaid diagrams.")
    else:
        print(f"✅ Rendered {count} diagram(s)")

#

# 6. UI Layout | GPT-4o Version

In [ ]:


title = widgets.HTML("<h2>🛠️ ADD + Mermaid Generator (JS Fixed)</h2>")
desc_input = widgets.Textarea(
    placeholder="E.g., A ride-sharing app that connects drivers with passengers...",
    layout={'width': '98%', 'height': '80px'}
)
qa_input = widgets.Text(
    placeholder="E.g., Availability, Low Latency, Scalability...",
    layout={'width': '98%'}
)
btn_go = widgets.Button(
    description="🚀 Generate Architecture",
    button_style='primary',
    layout={'width': '220px'}
)
output = widgets.Output(layout={'border': '1px solid #ccc', 'padding': '15px', 'margin_top': '10px'})

def on_go_clicked(b):
    with output:
        clear_output()

        if not desc_input.value.strip() or not qa_input.value.strip():
            print("⚠️ Please fill in all fields.")
            return

        # 1. Generate ADD
        display(Markdown("### 📐 Step 1: Generating ADD Architecture..."))
        add_text, add_prompt = generate_add_design(desc_input.value, qa_input.value)
        if debug_checkbox.value: display_collapsible("Prompt", add_prompt, "prompt")
        display(Markdown(add_text))
        display(Markdown("---"))

        # 2. Generate Mermaid Diagrams
        display(Markdown("### 📊 Step 2: Generating Diagrams..."))
        mermaid_text, mermaid_prompt = generate_mermaid_diagrams(desc_input.value, qa_input.value)
        if debug_checkbox.value: display_collapsible("Prompt", mermaid_prompt, "prompt")
        process_mermaid_response(mermaid_text)

btn_go.on_click(on_go_clicked)

display(widgets.VBox([
    title,
    widgets.Label("📝 System Description:"),
    desc_input,
    widgets.Label("⚡ Quality Attributes (comma-separated):"),
    qa_input,
    debug_checkbox,
    widgets.HBox([btn_go], layout={'justify_content': 'center', 'margin': '15px 0'}),
    output
], layout={'padding': '20px', 'max_width': '900px', 'margin': 'auto'}))